# Modeling : XGBoost

In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb
import xgboost as xgb


In [2]:
# PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
# prices = pd.read_csv("../datasets/sell_prices.csv", dtype = PRICE_DTYPES)
# for col, col_dtype in PRICE_DTYPES.items():
#     if col_dtype == "category":
#         prices[col] = prices[col].cat.codes.astype("int16")  # 类似于labelencoder，将store_id和item_id数字化表示
#         prices[col] -= prices[col].min()  
# # prices.head(1)

In [3]:
# CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
#          "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
#         "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
# cal = pd.read_csv("../datasets/calendar.csv", dtype = CAL_DTYPES)
# cal["date"] = pd.to_datetime(cal["date"])  #  转化为datetime类型
# for col, col_dtype in CAL_DTYPES.items():
#     if col_dtype == "category":
#         cal[col] = cal[col].cat.codes.astype("int16")  # 将event_name_1/event_type_1/event_name_2/event_type_2weekday数字化表示
#         cal[col] -= cal[col].min()  # NaN会转换为-1，该代表将NaN归为0，其他从1开始
# # cal.head(1)

In [4]:
# h = 28 
# max_lags = 57
# last_day = 1913  # 历史的最后一天
# fday = datetime(2016,4, 25)  # 预测的第一天
# fday
# nrows_demo=None
# # nrows_demo=10000

In [5]:
# %time
# first_day = 1850
# numcols = [f"d_{day}" for day in range(first_day, last_day + 1)]  # 选取历史数据，从first_day到tr_last（包含这一天）d_1200,d_1201
# catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
# dtype = {numcol: "float32" for numcol in numcols}   # d_1200为float32

# dtype.update({col: "category" for col in catcols if col != "id"})  # 除id外，都为category类型
# dt = pd.read_csv("../datasets/sales_train_validation.csv", usecols = catcols + numcols, dtype = dtype,nrows=nrows_demo)  # 取catcols和numcols的列(只取了需要的历史天数销量数据)
# dt

In [6]:
# for col in catcols:
#         if col != "id":
#             dt[col] = dt[col].cat.codes.astype("int16")  # 数字化
#             dt[col] -= dt[col].min()
    
    
# for day in range(last_day + 1, last_day + 28 +1):  # 遍历预测的每一天
#     dt[f"d_{day}"] = np.nan  # 先填充为NaN

# dt = pd.melt(dt,
#               id_vars = catcols,
#               value_vars = [col for col in dt.columns if col.startswith("d_")],
#               var_name = "d",
#               value_name = "sales")



In [7]:
# dt = dt.merge(cal, on= "d", copy = False)
# dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

In [8]:
# dt.head()

In [9]:
# FIRST_DAY = 1750

In [10]:
# dt["d"]=dt["d"].apply(lambda x: int(x.split("_")[1])) # d_1 to 1


In [11]:
IS_LOCAL_DEMO = True
# IS_LOCAL_DEMO = False

In [12]:
max_lags = 57
last_day = 1913  
fday = datetime(2016,4, 25) 

In [13]:
data_file = ("train_data.csv.gz" if(IS_LOCAL_DEMO) else "train_data.csv")
dt = pd.read_csv(data_file)

print("loading:",data_file)

loading: train_data.csv.gz


In [14]:
dt["date"] = pd.to_datetime(dt["date"])  


In [15]:
df = dt.copy()

In [16]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):  # [(7, lag_7), (28, lag_28)]
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)  

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean()) 

    date_features = {"wday": "weekday", "week": "weekofyear", "month": "month","quarter": "quarter",  "year": "year","mday": "day" }
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [17]:
create_fea(df)

In [18]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1850,0.0,2016-02-21,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,7,1,21
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1851,4.0,2016-02-22,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,8,1,22
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1852,0.0,2016-02-23,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,8,1,23
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1853,1.0,2016-02-24,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,8,1,24
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,1854,0.0,2016-02-25,11604,...,8.26,NaN,NaN,NaN,NaN,NaN,NaN,8,1,25


In [19]:
df.dropna(inplace = True)
df.shape

(90000, 31)

In [20]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 1_000, replace = False)  
print(len(fake_valid_inds))

1000


In [21]:
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)  
train_data = xgb.DMatrix(X_train.loc[train_inds] , label = y_train.loc[train_inds])
fake_valid_data = xgb.DMatrix(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],)


In [22]:
params = {'eta':0.1,
          'gamma':0.1,
          'max_depth':8, 
          'subsample':1,
          'lambda': 0.1,
          'alpha':0.1,
          'objective':'count:poisson', 
          'eval_metric':'rmse',
          'min_child_weight':1,
          'colsample_bytree':1,
          'colsample_bylevel':1,
          # 'scale_pos_weight':0.8, 
          # 'max_delta_step':0,
          # 'max_leaves': 0
          'seed':666,    
}  

In [23]:
watch_list = [(train_data,'train'), (fake_valid_data,'valid')]
m_xgb = xgb.train(params, train_data, num_boost_round=100, evals=watch_list, early_stopping_rounds=20) 

[0]	train-rmse:4.08967	valid-rmse:3.65727
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 20 rounds.
[1]	train-rmse:4.07823	valid-rmse:3.64495
[2]	train-rmse:4.06623	valid-rmse:3.63219
[3]	train-rmse:4.05363	valid-rmse:3.61869
[4]	train-rmse:4.04041	valid-rmse:3.60470
[5]	train-rmse:4.02656	valid-rmse:3.58982
[6]	train-rmse:4.01204	valid-rmse:3.57444
[7]	train-rmse:3.99686	valid-rmse:3.55849
[8]	train-rmse:3.98099	valid-rmse:3.54202
[9]	train-rmse:3.96442	valid-rmse:3.52466
[10]	train-rmse:3.94715	valid-rmse:3.50630
[11]	train-rmse:3.92910	valid-rmse:3.48755
[12]	train-rmse:3.91028	valid-rmse:3.46802
[13]	train-rmse:3.89064	valid-rmse:3.44765
[14]	train-rmse:3.87025	valid-rmse:3.42646
[15]	train-rmse:3.84903	valid-rmse:3.40458
[16]	train-rmse:3.82693	valid-rmse:3.38236
[17]	train-rmse:3.80400	valid-rmse:3.35857
[18]	train-rmse:3.78023	valid-rmse:3.33489
[19]	train-rmse:3.75554	valid-rmse:3.31005
[20]	

In [24]:
train_cols

Index(['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'wday', 'month',
       'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'lag_28',
       'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28', 'week',
       'quarter', 'mday'],
      dtype='object')

In [25]:
if(IS_LOCAL_DEMO):
    alphas = [1] # for local demo
else:
    alphas = [1.035, 1.03, 1.025] #[1.015,1.25,1.03] #[1.0,1.15,1.01] #[1.0,1.15,1.01] # for kaggle

weights = [1 / len(alphas)] * len(alphas) 
print(weights)

[1.0]


In [26]:
sub = 0.
cols = [f"F{i}" for i in range(1,29)]

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):  # [(1.035, 0.333), (1.03, 0.333), (1.025, 0.333)]
    te = dt
    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)  
        print(icount, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()  
        create_fea(tst)
        tst = xgb.DMatrix(tst.loc[tst.date == day , train_cols]) # xgb
        te.loc[te.date == day, "sales"] = alpha * m_xgb.predict(tst) 

    te_sub = te.loc[te.date >= fday, ["id", "sales"]]
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]  # [F1,F2,F3,...F28]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()  
    te_sub.fillna(0., inplace = True) 
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)



0 2016-04-25 00:00:00
0 2016-04-26 00:00:00
0 2016-04-27 00:00:00
0 2016-04-28 00:00:00
0 2016-04-29 00:00:00
0 2016-04-30 00:00:00
0 2016-05-01 00:00:00
0 2016-05-02 00:00:00
0 2016-05-03 00:00:00
0 2016-05-04 00:00:00
0 2016-05-05 00:00:00
0 2016-05-06 00:00:00
0 2016-05-07 00:00:00
0 2016-05-08 00:00:00
0 2016-05-09 00:00:00
0 2016-05-10 00:00:00
0 2016-05-11 00:00:00
0 2016-05-12 00:00:00
0 2016-05-13 00:00:00
0 2016-05-14 00:00:00
0 2016-05-15 00:00:00
0 2016-05-16 00:00:00
0 2016-05-17 00:00:00
0 2016-05-18 00:00:00
0 2016-05-19 00:00:00
0 2016-05-20 00:00:00
0 2016-05-21 00:00:00
0 2016-05-22 00:00:00
0 1 1.0


In [27]:

sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)


In [28]:
print(sub.shape)
print(sub.columns)
sub.head(3)

(20000, 29)
Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object', name='F')


F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,1.090994,0.929188,0.912075,0.906108,1.117657,1.247018,1.133614,1.06077,0.964408,...,0.938978,1.212351,1.181581,1.027601,0.936118,0.932882,0.866921,0.954276,1.282673,1.333103
1,FOODS_1_001_CA_2_validation,1.265997,1.173489,0.748496,0.924590,0.893916,1.669555,1.268805,1.02175,0.952916,...,1.014048,1.536314,1.425557,1.114387,1.090836,1.088353,1.207588,1.227914,1.631232,1.510149
2,FOODS_1_001_CA_3_validation,0.958185,0.867534,0.925391,0.943741,0.921091,1.478409,0.993912,0.85888,0.839692,...,0.981107,1.267132,1.125304,0.876783,0.826820,0.898950,0.836534,0.872754,1.193248,1.145454


In [29]:
if(IS_LOCAL_DEMO):
    sub.to_csv("submission_xgb.csv.gz",compression='gzip',index=False)# for local demo
else:
    sub.to_csv("submission_xgb.csv",index=False) # for kaggle  


In [ ]:
* ## ridge kaggle result ：0.22776